# REST API a JSON

Při práci s REST API využijeme následující postup:

1. Najdeme API, ze kterého chceme data získat. U API potřebujeme znát jeho adresu.
2. Podíváme se, jestli je API volně přístupné či zda vyžaduje registraci. Registrace může být i zdarma, jejím účelem je snaha zabránit přetížení služby velkým množstvím dotazů (existují samozřejmě i placená API).
3. Podíváme se na to, jaká data API nabízí (a v jakém formátu) a pokusíme se v nich zorientovat.
4. Zpracujeme data pomocí Pythonu.

Využijeme data z webu [DataUSA.io](https://datausa.io/), konkrétně data o pozici [vývojář(ka) software](https://datausa.io/profile/soc/software-developers). Data bychom si mohli stáhnout ručně, ale to vyžaduje lidský zásah. API nám umožní automatickou aktualizaci dat, takže se do reportu budou automaticky stahovat nová data.

Web [DataUSA.io](https://datausa.io/) registraci nevažaduje, můžeme tedy začít API používat. K prohlídnutí formátu dat můžeme použít i webový prohlížeč. Pro Google Chrome je k dispozici doplněk [JSON Formatter
](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa?hl=en), který data zformátuje do přehlednější podoby. Prohlížeč Mozilla Firefox umí data hezky zformátovat automaticky. Další možností je použití online nástrojů jako [JSON Formatter](https://jsonformatter.curiousconcept.com/), do kterého překopírujeme textový výstup. (On-line nástroje bychom neměli používat pro citlivá data, např. data o zákaznících firmy).

Adresa API je: [https://wad.datausa.io/api/data?PUMS%20Occupation=151252&drilldowns=Gender&measure=Total%20Population,Total%20Population%20MOE%20Appx,Record%20Count&Record%20Count%3E=5](https://wad.datausa.io/api/data?PUMS%20Occupation=151252&drilldowns=Gender&measure=Total%20Population,Total%20Population%20MOE%20Appx,Record%20Count&Record%20Count%3E=5).


In [2]:
import requests
import json

response = requests.get("https://wad.datausa.io/api/data?PUMS%20Occupation=151252&drilldowns=Gender&measure=Total%20Population,Total%20Population%20MOE%20Appx,Record%20Count&Record%20Count%3E=5")
# Převedeme data na slovník
sw_gender_data = json.loads(response.text)

Data si před zpracováním můžeme uložit. To je praktické hlavně u větších objemů dat a pomalejších API. Většinou je rychlejší načíst data ze souboru na disku než z internetu. V některých případech může být navíc počet přístupů na API omezen.

In [4]:
with open("data_zapis.json", "w", encoding="utf-8") as soubor:
    json.dump(sw_gender_data, soubor, indent=4)

Při použití modulu `json` je *úplně jedno*, jestli data stahujeme z internetu nebo nahráváme ze souboru. Data se vždy převedou do stejné struktury (slovníku) a pracujeme se s nimi stejně. Stejně bychom se slovníkem pracovali i v případě, že bychom si jej zapsali do programu sami.

Soubor je zde: [data_zapis.json](data_zapis.json)

In [3]:
with open('data_zapis.json', encoding='utf-8') as soubor:
    sw_gender_data = json.load(soubor)

Strukturu si můžeme [zobrazit graficky](assets/JSON.drawio.svg).

Zkusme si nyní načíst některé hodnoty.

In [5]:
# Nultý slovník s daty
print(sw_gender_data["data"][0])

{'ID Gender': 1, 'Gender': 'Male', 'ID Year': 2020, 'Year': '2020', 'Total Population': 1268944, 'Total Population MOE Appx': 27602.637285017023, 'Record Count': 59010, 'PUMS Occupation': 'Software developers', 'ID PUMS Occupation': '151252', 'Slug PUMS Occupation': 'software-developers'}
2020


In [6]:
# Kterého roku se týká nultý slovník?
print(sw_gender_data["data"][0]["Year"])

2020


Dále můžeme zjistit například:
- Kolik lidí (klíč `Total Population`) máme v posledním záznamu?
- Kterého pohlaví se poslední záznam týká?
- Je taky dobré zkontrolovat, zda se nám tam nevloudilo nějaké jiné povolání. Jaké povolání (`PUMS Occupation`) je u předposledního záznamu?

Můžeme si vyzkoušet i načtení metadat. Jaký je zdroj informací, které máme?

Dále si zkusme data uložit v nějakém přehlednějším formátu, abychom z nich snaději vytvořili vizualizace (např. v Tableau nebo Power BI). Výstup by měl vypadat jako tabulka níže. Využijeme formát CSV, tj. jednotlivé hodnoty oddělíme čárkou.

|Gender|Year|People |
|------|----|-------|
|Male  |2020|1268944|
|Female|2020|295706 |
|Male  |2019|1145155|
|Female|2019|267806 |
|Male  |2018|1029080|
|Female|2018|238581 |

In [9]:
data_zapis = []
for item in sw_gender_data["data"]:
    radek_zapis = [item["Gender"], item["Year"], str(item["Total Population"])]
    data_zapis.append(radek_zapis)

with open("data_zapis.csv", "w", encoding="utf-8") as soubor:
    zahlavi = "Gender,Year,People"
    print(zahlavi, file=soubor)
    for radek_zapis in data_zapis:
        retezec = ",".join(radek_zapis)
        print(retezec, file=soubor)

Pojďme si zkusit zapsat data ve formátu JSON. Budeme chtít zapsat data do dvourozměrného slovníku, kde jako klíče využijeme roky a jako klíče vnořených slovníků budou pohlaví.

```json
{
    "2020": {
        "Male": 1268944,
        "Female": 295706
    },
    "2019": {
        "Male": 1145155,
        "Female": 267806
    },
    "2018": {
        "Male": 1029080,
        "Female": 238581
    }
}
```

In [11]:
data_zapis = {}
for item in sw_gender_data["data"]:
    year = item["Year"]
    gender = item["Gender"]
    if year not in data_zapis:
        data_zapis[year] = {}
    data_zapis[year][gender] = item["Total Population"]

with open('data_zapis_2.json', "w", encoding='utf-8') as soubor:
    json.dump(data_zapis, soubor, indent=4)

Dále si můžeme vyzkoušet tabulku, ve které bude vývoj poměru žen a mužů v jednotlivých letech. Pro tvorbu tabulky můžeme využít data ve slovníku `data_zapis`.

|Year|Female percentage  |
|----|-------------------|
|2020|0.18899178730067429|
|2019|0.18953530918404685|
|2018|0.18820567959415016|

In [ ]:
female_percentage_list = []
for key, value in data_zapis.items():
    row_percentage = str(value["Female"] / (value["Male"] + value["Female"]))
    row = [key, row_percentage]
    female_percentage_list.append(row)


with open("data_zapis_2.csv", "w", encoding="utf-8") as soubor:
    zahlavi = "Year,Female percentage"
    print(zahlavi, file=soubor)
    for radek_zapis in female_percentage_list:
        retezec = ",".join(radek_zapis)
        print(retezec, file=soubor)
